<a href="https://colab.research.google.com/github/Aayush077/ML/blob/main/earthquake_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Earthquake Prediction


##Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
from google.colab import data_table
import matplotlib.pyplot as plt
!pip install cartopy
import cartopy.crs as ccrs
import seaborn as sns

##Importing dataset

In [ ]:
data_table.enable_dataframe_formatter()

df_raw = pd.read_csv('earthquake_1995-2023.csv')

display(df_raw.head())
display(df_raw.shape)

In [ ]:
print(df_raw.columns)


###Visualization of data

In [ ]:
plt.figure(figsize=(15,10))

ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global()

ax.coastlines()
ax.stock_img()
ax.add_feature(__import__("cartopy").feature.BORDERS)

plt.scatter(df_raw['longitude'], df_raw['latitude'],s=2, color='red',
    transform=ccrs.PlateCarree()
)

plt.title("All Affected Areas (Earthquake Locations)", fontsize=16)
plt.show()


In [ ]:
#Correlation HEATMAP
# Pick only the numeric columns and find how they relate
corr = df_raw.select_dtypes(include='number').corr()

# Draw the heatmap
plt.figure(figsize=(12,8))
sns.heatmap(corr, cmap='coolwarm', annot=True, fmt=".2f")
plt.title("Correlation Heatmap")
plt.show()


##Preprocessing & Spliting


###Shows information about your dataset, Creates a safe copy to work on

In [ ]:
print('Columns:', list(df_raw.columns))
print('\nDtypes:\n', df_raw.dtypes)
print('\nMissing values per column:\n', df_raw.isna().sum())

df = df_raw.copy()


In [ ]:
# clean and standardize column names – only rename what exists in your dataset
col_map = {
    'mag': 'magnitude',
    'magnitude': 'magnitude',
    'date_time': 'date_time',
    'time': 'time',
    'latitude': 'latitude',
    'longitude': 'longitude',
    'depth': 'depth',
    'magType': 'magType',
    'nst': 'num_stations',
    'dmin': 'dmin',
    'gap': 'gap',
    'rms': 'rms',
    'mmi': 'mmi',
    'cdi': 'cdi',
    'alert': 'alert',
    'tsunami': 'tsunami',
    'country': 'country',
    'continent': 'continent',
    'location': 'location',
    'title': 'title',
    'sig': 'sig'
}

# Apply renaming only to matching columns
df = df.rename(columns={col: col_map[col] for col in df.columns if col in col_map})

print("Renamed columns:")
print(df.columns.tolist())


In [ ]:
# Parse datetime column
if df.index.name == 'date_time':
    df = df.reset_index()

df['date_time'] = pd.to_datetime(df['date_time'], errors='coerce')

print("Invalid datetime rows:", df['date_time'].isna().sum())

# Create simple time features
df['year'] = df['date_time'].dt.year
df['month'] = df['date_time'].dt.month
df['day'] = df['date_time'].dt.day
df['hour'] = df['date_time'].dt.hour

# Set datetime as index
df = df.set_index('date_time')


In [ ]:
# Select useful features
feature_list = [
    'latitude','longitude','depth','num_stations','dmin','gap','rms',
    'mmi','cdi','tsunami','sig','year','month','hour',
    'magType','alert','continent','country','location','title'
]

# Keep only columns that exist
keep_cols = [c for c in feature_list if c in df.columns]
print("Keeping columns:", keep_cols)

# Build final modeling dataframe
df_model = df[keep_cols + ['magnitude']].copy()

# Drop rows with missing target
df_model = df_model.dropna(subset=['magnitude'])

# Fill missing values
num_cols = df_model.select_dtypes(include=['number']).columns.drop('magnitude')
df_model[num_cols] = df_model[num_cols].fillna(df_model[num_cols].median())

cat_cols = df_model.select_dtypes(include=['object']).columns
df_model[cat_cols] = df_model[cat_cols].fillna('missing')

print("Missing values after cleaning:\n", df_model.isna().sum())


### CORE PREPROCESSING AND SPLITING OF DATASET

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import joblib

# Train-test split
X = df_model.drop(columns=['magnitude'])
y = df_model['magnitude']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Train/Test shapes:", X_train.shape, X_test.shape)

# Identify numeric and categorical features
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

print("Numeric:", numeric_features)
print("Categorical:", categorical_features)

# Transformers
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(
    handle_unknown='ignore',
    sparse_output=False
)

# Build preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)


##LINEAR REG

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

# Linear Regression Pipeline
lr_model = Pipeline(steps=[
    ('pre', preprocessor),
    ('model', LinearRegression())
])

# Train
lr_model.fit(X_train, y_train)

# Predict
y_pred_lr = lr_model.predict(X_test)

# Evaluate
mse_lr = mean_squared_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)

print("Linear Regression MSE:", mse_lr)
print("Linear Regression R²:", r2_lr)

###Visualization(Actual Vs Predicted Linear Reg)

In [ ]:
y_pred_model = y_pred_lr

plt.figure(figsize=(8,6))
plt.scatter(y_test, y_pred_model, alpha=0.6, color='blue')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()],
         color='red', linestyle='--')

plt.title("Actual vs Predicted Magnitude (Linear Regression)")
plt.xlabel("Actual Magnitude")
plt.ylabel("Predicted Magnitude")
plt.grid(True)
plt.show()


## SV REG

In [ ]:
from sklearn.svm import SVR

# SVR Pipeline
svr_model = Pipeline(steps=[
    ('pre', preprocessor),
    ('model', SVR(kernel='rbf'))
])

# Train
svr_model.fit(X_train, y_train)

# Predict
y_pred_svr = svr_model.predict(X_test)

# Evaluate
mse_svr = mean_squared_error(y_test, y_pred_svr)
r2_svr = r2_score(y_test, y_pred_svr)

print("SVR MSE:", mse_svr)
print("SVR R²:", r2_svr)


###Visualization(Actual Vs Predicted SV Reg)

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(y_test, y_pred_svr, alpha=0.6, color='blue')
plt.plot([y_test.min(), y_test.max()],
         [y_test.min(), y_test.max()],
         color='red', linestyle='--')

plt.title("Actual vs Predicted Magnitude (SVR)")
plt.xlabel("Actual Magnitude")
plt.ylabel("Predicted Magnitude")
plt.grid(True)
plt.show()


##DECISION TREE REG

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# Decision Tree Pipeline
dt_model = Pipeline(steps=[
    ('pre', preprocessor),
    ('model', DecisionTreeRegressor(random_state=42))
])

# Train
dt_model.fit(X_train, y_train)

# Predict
y_pred_dt = dt_model.predict(X_test)

# Evaluate
mse_dt = mean_squared_error(y_test, y_pred_dt)
r2_dt = r2_score(y_test, y_pred_dt)

print("Decision Tree MSE:", mse_dt)
print("Decision Tree R²:", r2_dt)


In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(y_test, y_pred_dt, alpha=0.6, color='blue')
plt.plot([y_test.min(), y_test.max()],
         [y_test.min(), y_test.max()],
         color='red', linestyle='--')

plt.title("Actual vs Predicted Magnitude (Decision Tree)")
plt.xlabel("Actual Magnitude")
plt.ylabel("Predicted Magnitude")
plt.grid(True)
plt.show()


##Multi-Layer Perceptron

In [ ]:
from sklearn.neural_network import MLPRegressor

# Neural Network Pipeline (MLP)
mlp_model = Pipeline(steps=[
    ('pre', preprocessor),   # scaling + encoding
    ('model', MLPRegressor(
        hidden_layer_sizes=(128, 64, 32),
        activation='relu',
        solver='adam',
        max_iter=1500,
        random_state=42
    ))
])

# Train
mlp_model.fit(X_train, y_train)

# Predict
y_pred_mlp = mlp_model.predict(X_test)

# Evaluate
mse_mlp = mean_squared_error(y_test, y_pred_mlp)
r2_mlp = r2_score(y_test, y_pred_mlp)

print("Neural Network (MLP) MSE:", mse_mlp)
print("Neural Network (MLP) R²:", r2_mlp)


###Visualization(Actual Vs Predicted MLP)

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(y_test, y_pred_mlp, alpha=0.6, color='blue')
plt.plot([y_test.min(), y_test.max()],
         [y_test.min(), y_test.max()],
         color='red', linestyle='--')

plt.title("Actual vs Predicted Magnitude (Neural Network - MLP)")
plt.xlabel("Actual Magnitude")
plt.ylabel("Predicted Magnitude")
plt.grid(True)
plt.show()


## RANDOMFOREST REG

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Simple pipeline (no grid search)
rf_model = Pipeline([
    ("pre", preprocessor),("model", RandomForestRegressor(
        n_estimators=200, max_depth=20,random_state=42,n_jobs=-1))
])

# Fit model
rf_model.fit(X_train, y_train)

# Predict
y_pred_rf = rf_model.predict(X_test)

# Scores
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print(f"RandomForest MSE: {mse_rf:.4f}")
print(f"RandomForest R²: {r2_rf:.4f}")


### Visualization(Actual vs Predicted RandomForestReg)

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(y_test, y_pred_rf, alpha=0.6, color='blue')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()],
         color='red', linestyle='--')

plt.title("Actual vs Predicted Magnitude (RandomForest)")
plt.xlabel("Actual Magnitude")
plt.ylabel("Predicted Magnitude")
plt.grid(True)
plt.show()


In [ ]:
plt.plot(y_test.index[:20], y_test[:20], color='blue', label='Actual Magnitude')
plt.plot(y_test.index[:20], y_pred_rf[:20], color='orange', label='Predicted Magnitude')
plt.xlabel('Index')
plt.ylabel('Magnitude')
plt.title('Actual vs. Predicted Line Plot')
plt.legend()
plt.show()

###Top Feature Importances (RandomForest)

In [ ]:
# Extract the trained RandomForest model
rf = rf_model.named_steps["model"]

# Get encoded categorical feature names
ohe = rf_model.named_steps["pre"].named_transformers_["cat"]
cat_feature_names = ohe.get_feature_names_out(categorical_features).tolist()

# Combine with numeric feature names
all_feature_names = numeric_features + cat_feature_names

# Create importance dataframe
importances = pd.DataFrame({
    "feature": all_feature_names,
    "importance": rf.feature_importances_
})

# Sort and plot top 20
top_features = importances.sort_values("importance", ascending=False).head(20)

plt.figure(figsize=(10,6))
plt.barh(top_features["feature"], top_features["importance"], color="purple")
plt.gca().invert_yaxis()
plt.title("Top 20 Important Features (RandomForest)")
plt.xlabel("Importance Score")
plt.show()


## XG BOOST REG

In [ ]:
from xgboost import XGBRegressor

xgb_model = Pipeline([
        ("pre", preprocessor),
        ("model", XGBRegressor( objective='reg:squarederror',n_estimators=200,
            max_depth=6,learning_rate=0.1, random_state=42, n_jobs=-1))
    ])

# Train
xgb_model.fit(X_train, y_train)

# Predict
y_pred_xgb = xgb_model.predict(X_test)

# Scores
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

print(f"XGBoost MSE: {mse_xgb:.4f}")
print(f"XGBoost R²: {r2_xgb:.4f}")

### Visualization(Actual Vs Predicted XGBoostReg)

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(y_test, y_pred_xgb, alpha=0.6, color='green')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()],
         color='red', linestyle='--')

plt.title("Actual vs Predicted Magnitude (XGBoost)")
plt.xlabel("Actual Magnitude")
plt.ylabel("Predicted Magnitude")
plt.grid(True)
plt.show()


###Top Feature Importances (SVR)

In [ ]:
    # Extract the trained XGBoost model
    xgb = xgb_model.named_steps["model"]

    # Get feature names (numeric + encoded categorical)
    ohe = xgb_model.named_steps["pre"].named_transformers_["cat"]
    cat_feature_names = ohe.get_feature_names_out(categorical_features).tolist()

    all_feature_names = numeric_features + cat_feature_names

    # Get importance scores
    importances = pd.DataFrame({
        "feature": all_feature_names,
        "importance": xgb.feature_importances_
    })

    # Sort top 20
    top_features = importances.sort_values("importance", ascending=False).head(20)

    # Plot
    plt.figure(figsize=(10,6))
    plt.barh(top_features["feature"], top_features["importance"], color="green")
    plt.gca().invert_yaxis()
    plt.title("Top 20 Important Features (XGBoost)")
    plt.xlabel("Importance Score")
    plt.show()


##RESULTS

In [ ]:
# Evaluation Summary

results = []
results.append({"Model": "Linear Regression", "R2": r2_lr, "MSE": mse_lr})
results.append({"Model": "SVR", "R2": r2_svr, "MSE": mse_svr})
results.append({"Model": "Decision Tree", "R2": r2_dt, "MSE": mse_dt})
results.append({"Model": "Neural Network (MLP)", "R2": r2_mlp, "MSE": mse_mlp})
results.append({"Model": "Random Forest", "R2": r2_rf, "MSE": mse_rf})
results.append({"Model": "XGBoost", "R2": r2_xgb, "MSE": mse_xgb})

res_df = pd.DataFrame(results)
display(res_df)



In [ ]:
# Residuals (errors)
rf_residuals = y_test - y_pred_rf

plt.figure(figsize=(10,5))
plt.hist(rf_residuals, bins=25, color='cornflowerblue', edgecolor='black')
plt.title("Residual Distribution (RandomForest)")
plt.xlabel("Error (Actual - Predicted)")
plt.ylabel("Frequency")
plt.axvline(0, color='red', linestyle='--')
plt.show()


In [ ]:

xgb_residuals = y_test - y_pred_xgb
plt.figure(figsize=(10,5))
plt.hist(xgb_residuals, bins=25, color='seagreen', edgecolor='black')
plt.title("Residual Distribution (XGBoost)")
plt.xlabel("Error (Actual - Predicted)")
plt.ylabel("Frequency")
plt.axvline(0, color='red', linestyle='--')
plt.show()



In [ ]:
df_model.to_csv("earthquake_cleaned.csv", index=True)
print("Cleaned dataset saved as earthquake_cleaned.csv")
